# Amazon Bedrock AgentCore Policy - Lambda 타겟 정책 적용

## 개요

이 노트북은 JWT 클레임을 사용하여 Lambda 타겟에 **Cedar 정책 적용**을 시연합니다.

### 사전 요구사항

먼저 **`01-Setup-Gateway-Lambda.ipynb`**를 실행하여 Gateway와 Cognito 리소스를 생성하세요.

### 학습 목표

- Policy Engine 생성 및 Gateway에 연결
- Cognito Lambda 트리거를 사용하여 JWT 토큰에 커스텀 클레임 추가
- 속성 기반 접근 제어(ABAC)를 위한 Cedar 정책 구현
- 부서 기반 및 그룹 기반 접근 제어 패턴 테스트

### 아키텍처

```
                                ┌───────────────────────┐
                                │  Policy Engine        │
                                │  (Cedar 정책)         │
                                └───────────┬───────────┘
                                            │ 연결됨
                                            ▼
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT 토큰   │  AgentCore Gateway    │             │   Lambda    │
│   Cognito       │────────────>│  + 정책 평가          │────────────>│   타겟      │
│   + Lambda      │  + 클레임   │                       │  허용 시    │   (도구)    │
└─────────────────┘             └───────────────────────┘             └─────────────┘
```

---

## Part 1: 환경 설정

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions from common folder
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
    display_test_result,
)
from common.gateway_utils import (
    attach_policy_engine_to_gateway,
)
from common.policy_utils import (
    get_policy_engine,
    create_cedar_policy,
    wait_for_policy_active,
    delete_policy,
    cleanup_existing_policies,
    ensure_policy_engine,
)
from common.cognito_utils import (
    create_lambda_function,
    configure_cognito_trigger,
)

print("✓ Libraries loaded")

### 설정 로드

In [ ]:
# Load configuration file
config_path = Path.cwd() / "gateway_config.json"

if not config_path.exists():
    raise FileNotFoundError(
        "gateway_config.json not found. Run 01-Setup-Gateway-Lambda.ipynb first."
    )

with open(config_path, "r") as f:
    CONFIG = json.load(f)

# Extract key values
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]
POLICY_ENGINE_ID = CONFIG.get("policy_engine_id")

print("✓ Configuration loaded")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  POLICY_ENGINE_ID: {POLICY_ENGINE_ID or 'Not set'}")

In [ ]:
# Initialize AWS clients
session = boto3.Session(region_name=REGION)

lambda_client = session.client("lambda")
cognito_client = session.client("cognito-idp")
iam_client = session.client("iam")
sts_client = session.client("sts")
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")

---

## Part 2: Policy Engine 설정

Policy Engine을 생성하고 Gateway에 연결합니다.

In [ ]:
# Ensure Policy Engine exists (create if missing)
POLICY_ENGINE_ID = ensure_policy_engine(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    create_if_missing=True
)

if not POLICY_ENGINE_ID:
    raise RuntimeError("Failed to create or find Policy Engine")

# Save Policy Engine ID to config
CONFIG["policy_engine_id"] = POLICY_ENGINE_ID
with open(config_path, "w") as f:
    json.dump(CONFIG, f, indent=2)
print(f"✓ Policy Engine ID saved to config")

In [ ]:
# Attach Policy Engine to Gateway
engine = get_policy_engine(policy_client, POLICY_ENGINE_ID)
policy_engine_arn = engine.get("policyEngineArn")

attach_policy_engine_to_gateway(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    policy_engine_arn=policy_engine_arn,
    mode="ENFORCE"
)

#### Gateway에 연결된 Policy Engine
![policy_engine.png](img/policy_engine.png)

In [ ]:
# Clean up existing policies for fresh start
cleanup_existing_policies(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    require_confirmation=False
)

# Track created policies for cleanup
CREATED_POLICIES = []

---

## Part 3: 테스트 시나리오 1 - 부서 기반 접근 제어

**finance** 부서의 사용자만 환불 도구를 사용할 수 있도록 합니다.

### Cedar 정책 패턴

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
};
```

> **참고**: `principal.hasTag()` 및 `principal.getTag()`로 JWT 토큰 클레임에 접근합니다.

### Step 3.1: Finance 부서 클레임을 위한 Lambda 설정

In [ ]:
print("=" * 70)
print("Test Scenario 1: Department-Based Access Control")
print("=" * 70)

# Configure Lambda to add finance department claims
claims_finance = {
    "department_name": "finance",
    "employee_level": "senior",
    "cost_center": "CC-1001",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_finance,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

configure_cognito_trigger(
    cognito_client=cognito_client,
    lambda_client=lambda_client,
    user_pool_id=USER_POOL_ID,
    lambda_arn=lambda_arn,
    region=REGION,
    account_id=ACCOUNT_ID
)

print("\n✓ Lambda configured with claims:")
print(json.dumps(claims_finance, indent=2))

### Step 3.2: 토큰에 커스텀 클레임이 포함되었는지 확인

In [ ]:
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

claims = decode_token(token)

print("🔍 Token Claims:")
print(f"  department_name: {claims.get('department_name', 'N/A')}")
print(f"  employee_level: {claims.get('employee_level', 'N/A')}")
print(f"  cost_center: {claims.get('cost_center', 'N/A')}")

if claims.get("department_name") == "finance":
    print("\n✓ Custom claims are in the token")
else:
    print("\n⚠️  Custom claims not found - check Lambda trigger configuration")

### Step 3.3: 부서 검증 정책 생성

In [ ]:
policy_name = f"dept_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
}};'''

policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow only finance department"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id)

### Step 3.4: 테스트 - Finance 부서 (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.1: department_name='finance'")
print("=" * 70)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-finance"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Finance department should be allowed")

### Step 3.5: 테스트 - Engineering 부서 (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.2: department_name='engineering'")
print("=" * 70)

# Update Lambda to engineering department
claims_engineering = {
    "department_name": "engineering",
    "employee_level": "senior",
    "cost_center": "CC-2001",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_engineering,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)
print("\n✓ Lambda updated to department_name='engineering'")

print("\n⏳ Waiting for Lambda change propagation...")
time.sleep(5)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
claims = decode_token(token)
print(f"\nToken department_name: {claims.get('department_name')}")

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-engineering"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "Engineering department should be denied")

---

## Part 4: 테스트 시나리오 2 - 그룹 기반 접근 제어

**admins** 그룹의 사용자만 환불 도구를 사용할 수 있도록 합니다.

### Cedar 정책 패턴

JWT에서 그룹은 문자열로 직렬화되므로 `like` 연산자를 사용합니다:

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
};
```

In [ ]:
# Clean up previous policies
print("=" * 70)
print("Test Scenario 2: Group-Based Access Control")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 4.1: 그룹 클레임을 포함한 Lambda 설정

In [ ]:
# Configure Lambda with admins group
claims_with_admins = {
    "groups": ["admins", "developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_with_admins,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

configure_cognito_trigger(
    cognito_client=cognito_client,
    lambda_client=lambda_client,
    user_pool_id=USER_POOL_ID,
    lambda_arn=lambda_arn,
    region=REGION,
    account_id=ACCOUNT_ID
)

print("\n✓ Lambda configured with groups:")
print(f"   groups: {claims_with_admins['groups']}")

### Step 4.2: 그룹 검증 정책 생성

In [ ]:
policy_name = f"groups_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
}};'''

policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow only admins group"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id)

### Step 4.3: 테스트 - Admins 그룹에 속한 사용자 (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.1: groups=['admins', 'developers', 'team-alpha']")
print("=" * 70)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-admins"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "User in 'admins' group should be allowed")

### Step 4.4: 테스트 - Admins 그룹이 없는 사용자 (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.2: groups=['developers', 'team-alpha'] (no admins)")
print("=" * 70)

# Update Lambda without admins group
claims_no_admins = {
    "groups": ["developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(
    lambda_client=lambda_client,
    iam_client=iam_client,
    claims=claims_no_admins,
    account_id=ACCOUNT_ID,
    user_pool_id=USER_POOL_ID
)

print("\n⏳ Waiting for Lambda change propagation...")
time.sleep(5)

token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
claims = decode_token(token)
print(f"\nToken groups: {claims.get('groups')}")

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-no-admins"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "User without 'admins' group should be denied")

---

## Part 5: 고급 패턴

### 복합 조건 (부서 AND 금액 제한)

In [ ]:
print("\nAdvanced Pattern: Combined Conditions")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance" &&
    context.input.amount <= 1000
}};'''

print("Cedar Policy:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\nThis policy allows when:")
print("  ✓ User is in finance department")
print("  ✓ AND refund amount <= $1000")

### 패턴 매칭 (`like` 연산자)

| 패턴 | 매칭 |
|------|------|
| `"*admin*"` | "admin" 포함 |
| `"admin*"` | "admin"으로 시작 |
| `"*admin"` | "admin"으로 끝남 |
| `"team-*"` | "team-"으로 시작 |

In [ ]:
print("\nAdvanced Pattern: Team-Based Access with Wildcard")
print("=" * 70)

team_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*team-finance*"
}};'''

print("Cedar Policy:")
print("-" * 60)
print(team_cedar)
print("-" * 60)
print("\nMatches groups containing 'team-finance':")
print("  ✓ ['team-finance', 'developers']")
print("  ✓ ['admins', 'team-finance-leads']")

---

## Part 6: 정리

테스트 중 생성된 정책을 삭제합니다.

In [ ]:
print("=" * 70)
print("Cleanup")
print("=" * 70)

print(f"\nDeleting {len(CREATED_POLICIES)} policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)

CREATED_POLICIES.clear()
print("\n✓ Cleanup complete")

---

## 결론

이 튜토리얼에서 다룬 내용:

✅ Policy Engine 생성 및 Gateway에 연결  
✅ Cognito Lambda 트리거를 사용하여 JWT에 커스텀 클레임 추가  
✅ 속성 기반 접근 제어를 위한 Cedar 정책 구현  
✅ 부서 기반 접근 제어 (`principal.getTag("department_name")`)  
✅ 패턴 매칭을 사용한 그룹 기반 접근 제어 (`like "*admins*"`)  
✅ 복잡한 접근 제어 시나리오를 위한 복합 조건  

### 핵심 Cedar 문법 패턴

| 클레임 유형 | Cedar 문법 | 예시 |
|-------------|-----------|------|
| 문자열 (정확한 일치) | `principal.getTag("claim") == "value"` | `department_name == "finance"` |
| 문자열/배열 (포함) | `principal.getTag("claim") like "*value*"` | `groups like "*admins*"` |
| 입력값 검증 | `context.input.field <= value` | `context.input.amount <= 1000` |

> **참고**: JWT 배열 클레임은 문자열로 직렬화되므로 멤버십 확인에는 `like` 연산자를 사용합니다.